# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""16427590586955…",1.2927e19,"""13748672593630…","""../../../../da…","""53137716125203…","""17552472426760…",false
"""11067861697039…",1.4856e19,"""17509456300894…","""../../../../da…","""53137716125203…","""74957852055780…",true
"""89387067670103…",5.6458e17,"""17901602127942…","""../../../../da…","""17116018937744…","""17552472426760…",false
"""11067861697039…",4.0723e18,"""36496892364288…","""../../../../da…","""16242725016215…","""17552472426760…",false
"""16427590586955…",1.4677e19,"""13827620911066…","""../../../../da…","""16242725016215…","""91308170353926…",true
"""89387067670103…",1.6330e19,"""13772735269729…","""../../../../da…","""67235646795367…",null,false
"""89387067670103…",8.7675e18,"""55761170721680…","""../../../../da…","""16242725016215…","""91308170353926…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_MaritalStatus': 'PREDICTOR_0',
 'Customer_CLV': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""49095758556706…",1.2927e19,"""12836895089036…","""../../../../da…","""FirstMortgage3…","""13429522321942…","""Rejected"""
"""14756013941003…",1.4856e19,"""14626237665804…","""../../../../da…","""FirstMortgage3…","""44275034756552…","""Accepted"""
"""15945673584333…",5.6458e17,"""56233818150617…","""../../../../da…","""MoneyMarketSav…","""13429522321942…","""Rejected"""
"""14756013941003…",4.0723e18,"""65307222764780…","""../../../../da…","""BasicChecking""","""13429522321942…","""Rejected"""
"""49095758556706…",1.4677e19,"""14966437608300…","""../../../../da…","""BasicChecking""","""15132669479464…","""Accepted"""
"""15945673584333…",1.6330e19,"""56397629972255…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""15945673584333…",8.7675e18,"""13889055274020…","""../../../../da…","""BasicChecking""","""15132669479464…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""18415348268918…",1.2927e19,"""93161623034413…","""../../../../da…","""FirstMortgage3…","""90154055149000…","""Rejected"""
"""28969736852937…",1.4856e19,"""15319297043435…","""../../../../da…","""FirstMortgage3…","""34907635140506…","""Accepted"""
"""11008767978770…",5.6458e17,"""13745861662570…","""../../../../da…","""MoneyMarketSav…","""90154055149000…","""Rejected"""
"""28969736852937…",4.0723e18,"""38590593725286…","""../../../../da…","""BasicChecking""","""90154055149000…","""Rejected"""
"""18415348268918…",1.4677e19,"""12704976004551…","""../../../../da…","""BasicChecking""","""10903815599533…","""Accepted"""
"""11008767978770…",1.6330e19,"""10119642504385…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""11008767978770…",8.7675e18,"""17725447378680…","""../../../../da…","""BasicChecking""","""10903815599533…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_MaritalStatus=PREDICTOR_0
Customer_CLV=PREDICTOR_1
Customer_City=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""13102330982275…",1.2927e19,"""96251156408660…","""FirstMortgage3…","""92823927137138…","""Rejected"""
"""15401710789418…",1.4856e19,"""13141528402037…","""FirstMortgage3…","""11275270726603…","""Accepted"""
"""56129870482634…",5.6458e17,"""14494103835297…","""MoneyMarketSav…","""92823927137138…","""Rejected"""
"""15401710789418…",4.0723e18,"""56352048487634…","""BasicChecking""","""92823927137138…","""Rejected"""
"""13102330982275…",1.4677e19,"""16960025976366…","""BasicChecking""","""32447661303931…","""Accepted"""
"""56129870482634…",1.6330e19,"""43340770962786…","""UPlusFinPerson…",null,"""Rejected"""
"""56129870482634…",8.7675e18,"""17993594360527…","""BasicChecking""","""32447661303931…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'